In [1]:
from gurobi import *
import numpy as np
import pandas as pd
from scipy.spatial import distance
from itertools import chain, combinations

# PDSTSP


In [2]:
class Data:
    def __init__(self):
        self.customerNum = 0 
        self.nodeNum     = 0 
        self.droneNum    = 2
        self.cities      = []
        self.cor_X       = [] 
        self.cor_Y       = [] 
        self.serviceTime = [] 
        self.disMatrix   = [[]]
        self.dt          = None
        self.i_pot = None
        self.cus_can_served_by_drone = None
        self.drone_distances = None
        self.truck_distances = None
        
      
        

    def readData(self, path):
        self.dt = pd.read_csv(path, header = None).to_numpy()[:-1]
        self.customerNum = len(self.dt)
        self.i_pot = self.dt[0, 1:3]
        self.nodeNum = self.customerNum + 2 
        
        self.cities = [self.dt[i, 0] for i in range(len(self.dt))]
        
        self.cus_can_served_by_drone = [i for i in range(len(self.dt)) if self.dt[i, 3] == 0]
        
        self.drone_distances = [distance.euclidean((self.dt[i, 1:3]), self.i_pot)
                                if self.dt[i, 3] == 0 else float('inf')
                                for i in range(len(self.dt))]
        self.truck_distances = [[distance.cityblock(self.dt[i, 1:3], self.dt[j, 1:3])
                                 for i in range(len(self.dt))] for j in range(len(self.dt))]
    
    

        
       


In [3]:
data = Data()

data.readData("20140813T111857.csv")
print(data.dt)

print(data.customerNum)

[[ 0.   0.   0.   0.4]
 [ 1.   2.1  2.4  0. ]
 [ 2.   3.8  0.6  1. ]
 [ 3.   0.3  6.6  0. ]
 [ 4.   0.4  3.2  0. ]
 [ 5.   0.6  3.1  0. ]
 [ 6.   2.3 11.9  0. ]
 [ 7.   0.3  9.9  0. ]
 [ 8.   0.8  2.5  0. ]
 [ 9.   3.6 10.7  0. ]]
10


In [4]:
print(data.dt[1, 1:3])

[2.1 2.4]


In [5]:
print(data.cities)

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]


In [6]:
print(data.truck_distances)

[[0.0, 4.5, 4.3999999999999995, 6.8999999999999995, 3.6, 3.7, 14.2, 10.200000000000001, 3.3, 14.299999999999999], [4.5, 0.0, 3.4999999999999996, 5.999999999999999, 2.5000000000000004, 2.2, 9.7, 9.3, 1.4000000000000001, 9.799999999999999], [4.3999999999999995, 3.4999999999999996, 0.0, 9.5, 6.0, 5.699999999999999, 12.8, 12.8, 4.9, 10.299999999999999], [6.8999999999999995, 5.999999999999999, 9.5, 0.0, 3.4999999999999996, 3.7999999999999994, 7.300000000000001, 3.3000000000000007, 4.6, 7.4], [3.6, 2.5000000000000004, 6.0, 3.4999999999999996, 0.0, 0.30000000000000004, 10.6, 6.8, 1.1, 10.7], [3.7, 2.2, 5.699999999999999, 3.7999999999999994, 0.30000000000000004, 0.0, 10.5, 7.1000000000000005, 0.8000000000000002, 10.6], [14.2, 9.7, 12.8, 7.300000000000001, 10.6, 10.5, 0.0, 4.0, 10.9, 2.5000000000000013], [10.200000000000001, 9.3, 12.8, 3.3000000000000007, 6.8, 7.1000000000000005, 4.0, 0.0, 7.9, 4.1], [3.3, 1.4000000000000001, 4.9, 4.6, 1.1, 0.8000000000000002, 10.9, 7.9, 0.0, 11.0], [14.2999999

In [7]:
print(data.drone_distances)

[inf, 3.189043743820395, inf, 6.606814663663572, 3.22490309931942, 3.157530680769389, 12.120231020900551, 9.904544411531507, 2.6248809496813377, 11.289375536317321]


In [8]:
model = Model("PDSTSP")

Academic license - for non-commercial use only - expires 2021-07-04
Using license file /home/quanghuy205/gurobi.lic


In [9]:
#SET
N = [i for i in range (1, data.customerNum)]
G = [0] + N
M = [m for m in range(data.droneNum)]

N_d = data.cus_can_served_by_drone
N_t = [i for i in N if i not in N_d]
A = [(i,j) for i in G for j in G if i != j]
C_truck = {(i,j): data.truck_distances[i][j] for i,j in A}
C_drone = data.drone_distances

#Decision variables
# z_i = 0: if cus i is visited by vehicle, = 1 if visited by drones
# x_ij if (i->j) in vehicle tour
# # y_im = 1 if cus i assigned to drone m ()
z = [i for i in G]
x = [[[] for i in G] for j in G]  
y = [[[] for i in G] for m in M] 




In [10]:
        
#completion time
T = model.addVar(0, GRB.INFINITY, 1.0, GRB.CONTINUOUS, "traveltime")
expr = LinExpr(0)
expr.addTerms(1.0, T)
model.setObjective(expr, GRB.MINIMIZE)
expr.clear()

In [11]:
for i in G:
    for j in range(i, data.customerNum):
        if i != j:
            x[i][j] = model.addVar(0.0, 1.0, 0.0, GRB.BINARY)
            x[j][i] = x[i][j]
            expr = LinExpr(0)
            expr.addTerms(data.truck_distances[i][j], x[i][j])
        else:
            x[i][i] = model.addVar(0.0, 0.0, 0.0, GRB.BINARY)

model.addConstr(T >= expr, "truckTime")
expr.clear()

In [12]:
for m in M:
    expr = LinExpr(0)
    for i in G:
        if i in N_d:
            y[m][i] = model.addVar(0.0, 1.0, 0.0, GRB.BINARY)
            expr.addTerms(data.drone_distances[i], y[m][i])
        else:
            y[m][i] = model.addVar(0.0, 0.0, 0.0, GRB.BINARY)
        
    model.addConstr(T >= expr, "dronetime")
    expr.clear()
    
    

In [13]:
for j in range(1, data.customerNum):
    expr = LinExpr(0)
    for i in G:
        if (i != j):
            expr.addTerms(1.0, x[i][j])
    if j in N_d:
        for m in M:
            expr.addTerms(1.0, y[m][j])
    model.addConstr(expr == 1, "served customer once")
    expr.clear()


        


In [14]:
expr = LinExpr(0)
for i in range(1, data.customerNum):
    expr.addTerms(1.0, x[0][i])
    
model.addConstr(expr == 2, "deg-2 depot")
expr.clear()

In [15]:
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 13 rows, 76 columns and 126 nonzeros
Model fingerprint: 0x2cc351cb
Variable types: 1 continuous, 75 integer (75 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 12.1202309
Presolve removed 0 rows and 14 columns
Presolve time: 0.00s
Presolved: 13 rows, 62 columns, 126 nonzeros
Variable types: 1 continuous, 61 integer (61 binary)

Root relaxation: objective 0.000000e+00, 14 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    3   12.12023    0.00000   100%     -    0s
H    0     0                       3.1575307    0.00000   100

In [16]:
model.ObjVal

2.6248809496813377

In [17]:
model.printAttr('X')


    Variable            X 
-------------------------
  traveltime      2.62488 
          C2            1 
          C4            1 
         C25            1 
         C36            1 
         C49            1 
         C74            1 


In [18]:
model.getVars()

[<gurobi.Var traveltime (value 2.6248809496813377)>,
 <gurobi.Var C1 (value 0.0)>,
 <gurobi.Var C2 (value 1.0)>,
 <gurobi.Var C3 (value -0.0)>,
 <gurobi.Var C4 (value 1.0)>,
 <gurobi.Var C5 (value -0.0)>,
 <gurobi.Var C6 (value -0.0)>,
 <gurobi.Var C7 (value -0.0)>,
 <gurobi.Var C8 (value -0.0)>,
 <gurobi.Var C9 (value -0.0)>,
 <gurobi.Var C10 (value -0.0)>,
 <gurobi.Var C11 (value 0.0)>,
 <gurobi.Var C12 (value -0.0)>,
 <gurobi.Var C13 (value -0.0)>,
 <gurobi.Var C14 (value -0.0)>,
 <gurobi.Var C15 (value -0.0)>,
 <gurobi.Var C16 (value -0.0)>,
 <gurobi.Var C17 (value -0.0)>,
 <gurobi.Var C18 (value -0.0)>,
 <gurobi.Var C19 (value -0.0)>,
 <gurobi.Var C20 (value 0.0)>,
 <gurobi.Var C21 (value -0.0)>,
 <gurobi.Var C22 (value -0.0)>,
 <gurobi.Var C23 (value -0.0)>,
 <gurobi.Var C24 (value -0.0)>,
 <gurobi.Var C25 (value 1.0)>,
 <gurobi.Var C26 (value -0.0)>,
 <gurobi.Var C27 (value -0.0)>,
 <gurobi.Var C28 (value 0.0)>,
 <gurobi.Var C29 (value -0.0)>,
 <gurobi.Var C30 (value -0.0)>,
 <g